In [74]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


c:\Users\Lenovo\OneDrive - Universidad EAFIT\Documentos\Obsidian\Obsidian\2024-1\Cuántica\ps\notas\.venv\Scripts\python.exe: No module named pip


In [75]:
import numpy as np
from numpy import ndarray
from numpy import sqrt, log2,ceil
from typing import Callable

In [76]:
N = 5


def f(index):
    return index == 4 or index == 2


In [77]:
def H_otimes_n(n) -> ndarray:
    hadamard: ndarray = np.sqrt(0.5) * np.array([[1, 1], [1, -1]])
    result = hadamard
    n -= 1  # primera iteración siempre inicia en hadamard, pero no cuenta como multiplicación

    for _ in range(n):
        result = np.kron(result, hadamard)
    # Kronecker products crea submatrices, esto reshapea a matriz
    op = result.reshape(-1, result.shape[-1])
    return op

In [78]:
def phase_inversion(N, f:Callable)-> ndarray:
    data = list(-1 if f(x) else 1 for x in range(N))
    U_f = np.diag(data)
    return U_f

In [79]:
def proyection_about_mean(N):
    # Projector |0^n><0^n|
    projector = np.zeros((N, N))
    projector[0, 0] = 1
    # Identity matrix I
    identity = np.eye(N)

    # Grover diffusion operator: U_0^n = 2|0^n><0^n| - I
    reflection_about_mean = 2 * projector - identity
    return reflection_about_mean


In [80]:
def op_deets(op: ndarray, label: str):
    print()
    print(label)
    print(op.shape)
    print(op)
    print()
    print()

def is_op_unitary(A:ndarray, N: int):
    case1 = A.conj().T @ A
    case2 = A @ A.conj().T
    Identity = np.eye(N)
    assert np.allclose(case1, case2) 
    assert np.allclose(case2, Identity)

def is_v_unit(v:ndarray, m):
    v_2 = (v.T @ v)[0][0]
    tol = 1e-10
    is_approximate = tol > abs(v_2 - 1)
    if m:
        print("v_2 inner product")
        print(v_2)
    if not is_approximate:
        print("hay que dividir por", v_2 )
    assert is_approximate

def test_ops(n, N, f, psi, m:bool = False):
    if m:
        print(f"Qubits n: {n}, Estados totales N : {N}")
    H = H_otimes_n(n)
    if m:
        label = "Hadamard_otimes_n:"
        op_deets(H, label)
    is_op_unitary(H, N)
    # Hadamard además de unitaria, es Hermitian?
    assert np.allclose(H, H.conj().T)
    # P_I es una función oráculo
    P_I = phase_inversion(N, f)
    if m:
        label = "Phase inversion:"
        op_deets(P_I, label)
    is_op_unitary(P_I, N)

    is_v_unit(psi,m)
    if m:
        print("psi es unitario")
        print(psi)

    U_0n = proyection_about_mean(N)
    if m:
        label = "Proyection about mean:"
        op_deets(U_0n, label)
        is_op_unitary(U_0n, N)

  



In [83]:
def grover(NumberStates, f: Callable):
    n = int(ceil(log2(NumberStates)))
    N = int(np.power(2, n))
    H = H_otimes_n(n)
    ket_h = np.zeros((N, 1))
    ket_h[0][0] = 1
    ket_h = H @ ket_h
    test_ops(n, N, f, ket_h)
    reflect = phase_inversion(N, f)
    mean_reflection = proyection_about_mean(N)

    for i in range(int(sqrt(N))):
        ket_h = reflect @ ket_h

        ket_h = H @ ket_h

        ket_h = mean_reflection @ ket_h

        ket_h = H @ ket_h

    return ket_h


grover(N, f)


array([[-0.35355339],
       [-0.35355339],
       [ 0.35355339],
       [-0.35355339],
       [ 0.35355339],
       [-0.35355339],
       [-0.35355339],
       [-0.35355339]])

In [82]:
def result(N,f):
    psi = grover(N, f)
    bitmask = psi > 0
    # Filter the original array using boolean indexing and get the index as well
    found = list(psi[bitmask]), np.nonzero(bitmask)[0].tolist()
    return found[1]
result(N,f)

[2, 4]